In [32]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
from sklearn.model_selection import train_test_split
# warnings.filterwarnings('ignore')

Loading the data, removing nan values, normalizing columns with huge variance, one hot encoding categorical variables.

In [33]:
columns = ["Age", "Workclass", "Final Weight", "Education", "Education Number of Years", "Marital-status", "Occupation", "Relationship", "Race", "Gender", "Capital-gain", "Capital-loss", "Hours-per-week", "Native-country", "Income"]
df = pd.read_csv("C:/Users/Dell/Documents/Explainable_machine_learning_2024/datasets/datasets/adult/raw/adult.data", index_col=None, header=None, names=columns)
print(df.shape)
df.head(3)

(32561, 15)


,Age,Workclass,Final Weight,Education,Education Number of Years,Marital-status,Occupation,Relationship,Race,Gender,Capital-gain,Capital-loss,Hours-per-week,Native-country,Income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K


In [34]:
qualitative_columns = ['Workclass', "Education", 'Marital-status', 'Occupation', "Relationship", 'Race', 'Gender', 'Native-country']
for col in qualitative_columns + ['Income']:
    print(col, ' unique values: ', df[col].unique())

Workclass  unique values:  [' State-gov' ' Self-emp-not-inc' ' Private' ' Federal-gov' ' Local-gov'
 ' ?' ' Self-emp-inc' ' Without-pay' ' Never-worked']
Education  unique values:  [' Bachelors' ' HS-grad' ' 11th' ' Masters' ' 9th' ' Some-college'
 ' Assoc-acdm' ' Assoc-voc' ' 7th-8th' ' Doctorate' ' Prof-school'
 ' 5th-6th' ' 10th' ' 1st-4th' ' Preschool' ' 12th']
Marital-status  unique values:  [' Never-married' ' Married-civ-spouse' ' Divorced'
 ' Married-spouse-absent' ' Separated' ' Married-AF-spouse' ' Widowed']
Occupation  unique values:  [' Adm-clerical' ' Exec-managerial' ' Handlers-cleaners' ' Prof-specialty'
 ' Other-service' ' Sales' ' Craft-repair' ' Transport-moving'
 ' Farming-fishing' ' Machine-op-inspct' ' Tech-support' ' ?'
 ' Protective-serv' ' Armed-Forces' ' Priv-house-serv']
Relationship  unique values:  [' Not-in-family' ' Husband' ' Wife' ' Own-child' ' Unmarried'
 ' Other-relative']
Race  unique values:  [' White' ' Black' ' Asian-Pac-Islander' ' Amer-Indian-Es

In [35]:
df_cleaned = df.replace(' ?', np.nan).dropna()

In [36]:
df_encoded = pd.get_dummies(df_cleaned, columns=qualitative_columns, drop_first=True, dtype = float)
df_encoded = df_encoded.replace(' <=50K', 0)
df_encoded = df_encoded.replace(' >50K', 1)
print(df_encoded.columns)

df_normalized = df_encoded.copy()
df_normalized['Final Weight'] = df_encoded['Final Weight'] / df_encoded['Final Weight'].max()
df_normalized['Capital-gain'] = df_encoded['Capital-gain'] / df_encoded['Capital-gain'].max()
# df_normalized['Final Weight'] = df_encoded['Final Weight'] / df_encoded['Final Weight'].max()

# Display the encoded DataFrame
print(df_normalized.head(1))
# count_sex = df['Gender'].value_counts()
# print(count_sex)

Index(['Age', 'Final Weight', 'Education Number of Years', 'Capital-gain',
       'Capital-loss', 'Hours-per-week', 'Income', 'Workclass_ Local-gov',
       'Workclass_ Private', 'Workclass_ Self-emp-inc',
       'Workclass_ Self-emp-not-inc', 'Workclass_ State-gov',
       'Workclass_ Without-pay', 'Education_ 11th', 'Education_ 12th',
       'Education_ 1st-4th', 'Education_ 5th-6th', 'Education_ 7th-8th',
       'Education_ 9th', 'Education_ Assoc-acdm', 'Education_ Assoc-voc',
       'Education_ Bachelors', 'Education_ Doctorate', 'Education_ HS-grad',
       'Education_ Masters', 'Education_ Preschool', 'Education_ Prof-school',
       'Education_ Some-college', 'Marital-status_ Married-AF-spouse',
       'Marital-status_ Married-civ-spouse',
       'Marital-status_ Married-spouse-absent',
       'Marital-status_ Never-married', 'Marital-status_ Separated',
       'Marital-status_ Widowed', 'Occupation_ Armed-Forces',
       'Occupation_ Craft-repair', 'Occupation_ Exec-managerial

In [43]:
print(df_normalized)

       Age  Final Weight  Education Number of Years  Capital-gain  \
0       39      0.052210                         13      0.021740   
1       50      0.056113                         13      0.000000   
2       38      0.145245                          9      0.000000   
3       53      0.158093                          7      0.000000   
4       28      0.227930                         13      0.000000   
...    ...           ...                        ...           ...   
32556   27      0.173302                         12      0.000000   
32557   40      0.103976                          9      0.000000   
32558   58      0.102317                          9      0.000000   
32559   22      0.135710                          9      0.000000   
32560   52      0.193929                          9      0.150242   

       Capital-loss  Hours-per-week  Income  Workclass_ Local-gov  \
0                 0              40       0                   0.0   
1                 0              

1. Training the first model, simple logistic regresion

In [83]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Assuming 'train' is your DataFrame
X = df_normalized.drop(columns=['Income'])  # Features
y = df_normalized['Income']  # Target variable

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a logistic regression model with L1 penalty (Lasso)
model = LogisticRegression(penalty='l1', solver='liblinear')  # 'liblinear' supports L1

# Fit the model
model.fit(X_train, y_train)

# Evaluate the model (optional)
accuracy = model.score(X_test, y_test)
print(f"Model accuracy: {accuracy:.4f}")


Model accuracy: 0.8538


2. We will look at race as a protected characteristic. We will assume that white people are privileged and other are not. 

In [39]:
# Creating 'white' DataFrames (rows where 'Race_White' == 1)
x_test_white = X_test[X_test['Race_ White'] == 1]
y_test_white = y_test[X_test['Race_ White'] == 1]

# Creating 'other' DataFrames (rows where 'Race_White' != 1)
x_test_other = X_test[X_test['Race_ White'] != 1]
y_test_other = y_test[X_test['Race_ White'] != 1]

# Optional: Print the sizes of the new DataFrames
print(f"x_test_white shape: {x_test_white.shape}, y_test_white shape: {y_test_white.shape}")
print(f"x_test_other shape: {x_test_other.shape}, y_test_other shape: {y_test_other.shape}")


x_test_white shape: (5176, 96), y_test_white shape: (5176,)
x_test_other shape: (857, 96), y_test_other shape: (857,)


In [61]:
predicted_y_white = model.predict(x_test_white)
predicted_y_other = model.predict(x_test_other)
print('accuracy in white class:', model.score(x_test_white, y_test_white))
print('accuracy in other class:', model.score(x_test_other, y_test_other))

accuracy in white class: 0.8458268933539412
accuracy in other class: 0.9008168028004667


In [64]:
def calculate_group_fairness(predicted_y_white, predicted_y_other):
    p_white = np.sum(predicted_y_white) / predicted_y_white.shape[0]
    p_other = np.sum(predicted_y_other) / predicted_y_other.shape[0]
    return p_white/p_other

def calculate_equal_oportunity(predicted_y_white, predicted_y_other, true_y_white, true_y_other):
    earn_more_white = predicted_y_white[true_y_white == 1]
    earn_more_other = predicted_y_other[true_y_other == 1]
    p_white = np.sum(earn_more_white) / earn_more_white.shape[0]
    p_other = np.sum(earn_more_other) / earn_more_other.shape[0]
    return p_white/p_other

def calculate_predictive_equality(predicted_y_white, predicted_y_other, true_y_white, true_y_other):
    earn_more_white = predicted_y_white[true_y_white == 0]
    earn_more_other = predicted_y_other[true_y_other == 0]
    p_white = np.sum(earn_more_white) / earn_more_white.shape[0]
    p_other = np.sum(earn_more_other) / earn_more_other.shape[0]
    return p_white/p_other

print('group fariness = ', calculate_group_fairness(predicted_y_white, predicted_y_other))
print('equal oportunity = ', calculate_equal_oportunity(predicted_y_white, predicted_y_other, y_test_white, y_test_other))
print('predictive_equality = ', calculate_predictive_equality(predicted_y_white, predicted_y_other, y_test_white, y_test_other))

group fariness =  1.740037709657107
equal oportunity =  1.0658998322549722
predictive_equality =  1.9039874749253876


3. We will train the second model, an xg boost model.

In [71]:
import xgboost as xgb
from sklearn.metrics import accuracy_score

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Set hyperparameters for the XGBoost model
params = {
    'objective': 'binary:logistic',  # For binary classification
    'eval_metric': 'logloss',        # Evaluation metric: Log loss
    'max_depth': 6,                  # Maximum depth of trees
    'eta': 0.3,                      # Learning rate
    'lambda': 1,                     # L2 regularization (Ridge)
    'alpha': 0,                      # L1 regularization (Lasso)
}

bst = xgb.train(params, dtrain, num_boost_round=100)

y_pred_proba = bst.predict(dtest)  # Predicted probabilities
y_pred = (y_pred_proba >= 0.5).astype(int)  # Convert to binary (0 or 1)

accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

dtest_white = xgb.DMatrix(x_test_white, label=y_test_white)
dtest_other = xgb.DMatrix(x_test_other, label=y_test_other)
xgb_predicted_white = bst.predict(dtest_white)
xgb_predicted_white = (xgb_predicted_white >= 0.5).astype(int)
xgb_predicted_other = bst.predict(dtest_other)
xgb_predicted_other = (xgb_predicted_other >= 0.5).astype(int)

Model Accuracy: 0.87


Calculate metrics

In [72]:
print('group fariness = ', calculate_group_fairness(xgb_predicted_white, xgb_predicted_other))
print('equal oportunity = ', calculate_equal_oportunity(xgb_predicted_white, xgb_predicted_other, y_test_white, y_test_other))
print('predictive_equality = ', calculate_predictive_equality(xgb_predicted_white, xgb_predicted_other, y_test_white, y_test_other))

group fariness =  1.6777949510561567
equal oportunity =  1.01522706762317
predictive_equality =  1.9042576973884768


4. We can see that the classes are weary unbalanced:

In [76]:
count_white = X_train['Race_ White'].value_counts()
print(count_white)

Race_ White
1.0    20757
0.0     3372
Name: count, dtype: int64


Lets train a model on a more balanced dataset

In [80]:
# Separate rows based on the 'Race_White' column in X_train
X_white_0 = X_train[X_train['Race_ White'] == 0]  # All rows with 'Race_White' == 0
X_white_1 = X_train[X_train['Race_ White'] == 1]  # All rows with 'Race_White' == 1

# Randomly sample from 'Race_White' == 1 to match the count of 'Race_White' == 0
X_white_1_sampled = X_white_1.sample(n=len(X_white_0), random_state=42)

# Combine the two DataFrames to create a balanced DataFrame
X_train_balanced = pd.concat([X_white_0, X_white_1_sampled], ignore_index=True)

# Create y_train_balanced corresponding to the balanced X_train
# Get indices of sampled rows for 'Race_White' == 1
indices_1_sampled = X_white_1_sampled.index

# Combine the corresponding y values for 'Race_White' == 0 and the sampled 'Race_White' == 1
y_train_balanced = pd.concat([y_train[X_train['Race_ White'] == 0], y_train.loc[indices_1_sampled]], ignore_index=True)

# Optional: Shuffle the rows for randomness
X_train_balanced = X_train_balanced.sample(frac=1, random_state=42).reset_index(drop=True)
y_train_balanced = y_train_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Print the shapes of the new DataFrames
print(f"Balanced X_train shape: {X_train_balanced.shape}")
print(f"Balanced y_train shape: {y_train_balanced.shape}")



Balanced X_train shape: (6744, 96)
Balanced y_train shape: (6744,)


In [82]:
# Create a logistic regression model with L1 penalty (Lasso)
model_balanced = LogisticRegression(penalty='l1', solver='liblinear')  # 'liblinear' supports L1

# Fit the model
model_balanced.fit(X_train_balanced, y_train_balanced)

# Evaluate the model (optional)
accuracy = model.score(X_test, y_test)
print(f"Model accuracy: {accuracy:.4f}")

Model accuracy: 0.8536


In [84]:
predicted_y_white_balanced = model_balanced.predict(x_test_white)
predicted_y_other_balanced = model_balanced.predict(x_test_other)

In [85]:
print('group fariness = ', calculate_group_fairness(predicted_y_white_balanced, predicted_y_other_balanced))
print('equal oportunity = ', calculate_equal_oportunity(predicted_y_white_balanced, predicted_y_other_balanced, y_test_white, y_test_other))
print('predictive_equality = ', calculate_predictive_equality(predicted_y_white_balanced, predicted_y_other_balanced, y_test_white, y_test_other))

group fariness =  1.7247069809376612
equal oportunity =  1.052329348279504
predictive_equality =  1.9110132589657027


5. All models bechave in very similar ways